In [110]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import ruptures as rpt
from datetime import datetime, date
from scipy.stats import mannwhitneyu
from matplotlib import pyplot as plt

In [2]:
politicians_path = '/Users/brahmaninutakki/saarland/twitter_extension/data/przemek/politicians'
user_ids = '/Users/brahmaninutakki/saarland/twitter_extension/data/przemek/data_users_ids'

#### check dict structure for user info in data_users_ids

In [3]:
temp_path = '/Users/brahmaninutakki/saarland/twitter_extension/data/przemek/data_users_ids/AdamKinzinger/1734316312793812531_UserByScreenName.json'

In [4]:
with open(temp_path, 'r') as f:
    temp_data = json.load(f)

In [5]:
k = temp_data.copy()

while type(k) is dict:
    print(k.keys())
    k = list(k.values())[0]

dict_keys(['data'])
dict_keys(['user'])
dict_keys(['result'])
dict_keys(['__typename', 'id', 'rest_id', 'affiliates_highlighted_label', 'has_graduated_access', 'is_blue_verified', 'profile_image_shape', 'legacy', 'smart_blocked_by', 'smart_blocking', 'legacy_extended_profile', 'is_profile_translatable', 'verification_info', 'business_account'])


In [6]:
del temp_data, k, temp_path

#### get mapping for user ids and names

In [7]:
user_id_maps = {}

for user_dir in tqdm(os.listdir(user_ids)):
    try:
        for json_path in os.listdir(f'{user_ids}/{user_dir}'):
            with open(f'{user_ids}/{user_dir}/{json_path}', 'r') as f:
                data = json.load(f)

            userid = data['data']['user']['result']['rest_id']

        user_id_maps[user_dir] = userid
    except Exception as e:
        print(user_dir, e)
        user_id_maps[user_dir] = 'None'
    


100%|██████████| 152/152 [00:00<00:00, 15457.62it/s]

kremlinrussia_e_lang=en 'user'


In [8]:
reverse_user_id_maps = {v: k for k, v in user_id_maps.items()}

#### check if all the dirs in the politicians file are in the mappings

In [9]:
for politician in os.listdir(politicians_path):
    if politician not in user_id_maps.values():
        print(politician)

.DS_Store


In [10]:
for politician in os.listdir(politicians_path):
    if politician == '.DS_Store':
        continue
    for fps in os.listdir(f'{politicians_path}/{politician}'):
        try:
            if fps.endswith('.json'):
                with open(f'{politicians_path}/{politician}/{fps}', 'r') as f:
                    data = json.load(f)

                for inst in data['data']['user']['result']['timeline_v2']['timeline']['instructions']:
                    if 'entry' in inst or 'entries' in inst:
                        continue
                    else:
                        if len(inst.keys()) > 1:
                            print(politician, fps, inst.keys())

        except Exception as e:
            print(politician, fps, e)

15007149 1734329776271538162_UserTweets.json 'result'
747807250819981312 1734369748994013722_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369763369054109_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369760309727210_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369737181964938_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369763127397224_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369755450842418_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369741073248038_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369758018823908_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369742681689398_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369763486435113_UserTweets.json Expecting value: l

In [12]:
politician_tweet_ids = {}
temp = []
for politician in os.listdir(politicians_path):
    politician_tweet_ids[politician] = []
    if politician == '.DS_Store':
        continue
    for fps in os.listdir(f'{politicians_path}/{politician}'):
        try:
            if fps.endswith('.json'):
                with open(f'{politicians_path}/{politician}/{fps}', 'r') as f:
                    data = json.load(f)

                for inst in data['data']['user']['result']['timeline_v2']['timeline']['instructions']:
                    if 'entry' in inst:
                        politician_tweet_ids[politician].append(inst['entry']['entryId'])
                    elif 'entries' in inst:
                        for entry in inst['entries']:
                            politician_tweet_ids[politician].append(entry['entryId'])

                if len(politician_tweet_ids[politician]) != len(set(politician_tweet_ids[politician])):
                    temp.append('duplicates')
                else:
                    temp.append('no_duplicates')

        except Exception as e:
            print(politician, fps, e)

set(temp)

15007149 1734329776271538162_UserTweets.json 'result'
747807250819981312 1734369748994013722_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369763369054109_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369760309727210_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369737181964938_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369763127397224_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369755450842418_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369741073248038_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369758018823908_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369742681689398_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369763486435113_UserTweets.json Expecting value: l

{'duplicates', 'no_duplicates'}

In [46]:
tdata = None
politician_tweet_data = {}
for politician in tqdm(os.listdir(politicians_path)):
    if politician == '.DS_Store':
        continue
    politician_tweet_data[politician] = {}

    for fps in os.listdir(f'{politicians_path}/{politician}'):
        try:
            if fps.endswith('.json'):
                with open(f'{politicians_path}/{politician}/{fps}', 'r') as f:
                    data = json.load(f)

                for inst in data['data']['user']['result']['timeline_v2']['timeline']['instructions']:
                    if 'entry' in inst:
                        if 'legacy' in inst['entry']['content']['itemContent']['tweet_results']['result']:
                            temp = inst['entry']['content']['itemContent']['tweet_results']['result']
                        elif 'tweet' in inst['entry']['content']['itemContent']['tweet_results']['result']:
                            temp = inst['entry']['content']['itemContent']['tweet_results']['result']['tweet']
                        
                        try:
                            view_count = temp['views']['count']
                        except:
                            view_count = -1

                        temp = temp['legacy']
                        
                        bookmark_count = temp['bookmark_count']
                        created_at = temp['created_at']
                        tweet_id = temp['id_str']
                        favorite_count = temp['favorite_count']
                        quote_count = temp['quote_count']
                        reply_count = temp['reply_count']
                        retweet_count = temp['retweet_count']

                        if tweet_id in politician_tweet_data[politician]:
                            pass
                        else:
                            temp_dict = {'created_at': created_at, 'view_count': view_count,
                                         'favorite_count': favorite_count,
                                        'retweet_count': retweet_count, 'reply_count': reply_count,
                                        'quote_count': quote_count, 'bookmark_count': bookmark_count}
                            politician_tweet_data[politician][tweet_id] = temp_dict

                    elif 'entries' in inst:
                        for entry in inst['entries']:
                            if 'itemContent' not in entry['content']:
                                continue
                            if 'legacy' not in entry['content']['itemContent']['tweet_results']['result']:
                                continue

                            temp = entry['content']['itemContent']['tweet_results']['result']

                            try:
                                view_count = temp['views']['count']
                            except:
                                view_count = -1

                            temp = temp['legacy']

                            bookmark_count = temp['bookmark_count']
                            created_at = temp['created_at']
                            tweet_id = temp['id_str']
                            favorite_count = temp['favorite_count']
                            quote_count = temp['quote_count']
                            reply_count = temp['reply_count']
                            retweet_count = temp['retweet_count']

                            if tweet_id in politician_tweet_data[politician]:
                                pass
                            else:
                                temp_dict = {'created_at': created_at, 'view_count': view_count,
                                             'favorite_count': favorite_count,
                                            'retweet_count': retweet_count, 'reply_count': reply_count,
                                            'quote_count': quote_count, 'bookmark_count': bookmark_count}
                                politician_tweet_data[politician][tweet_id] = temp_dict
        except Exception as e:
            print(politician, fps, e)
    # break

 19%|█▊        | 28/150 [00:01<00:05, 21.16it/s]

15007149 1734329776271538162_UserTweets.json 'result'


 77%|███████▋  | 115/150 [00:07<00:02, 15.30it/s]

747807250819981312 1734369748994013722_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369763369054109_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369760309727210_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369737181964938_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369763127397224_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369755450842418_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369741073248038_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369758018823908_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369742681689398_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 1734369763486435113_UserTweets.json Expecting value: line 1 column 1 (char 0)
747807250819981312 17343697434

 81%|████████▏ | 122/150 [00:07<00:01, 18.19it/s]

1168968080690749441 1734375282569596444_UserTweets.json 'timeline_v2'


 90%|█████████ | 135/150 [00:08<00:00, 18.62it/s]

25073877 1734410522258456086_UserTweets.json 'result'
25073877 1734410521634541926_UserTweets.json 'result'


100%|██████████| 150/150 [00:09<00:00, 16.10it/s]


In [47]:
politician_tweet_data['103065157']

{'1805989247264170273': {'created_at': 'Wed Jun 26 15:39:24 +0000 2024',
  'view_count': '28330',
  'favorite_count': 92,
  'retweet_count': 17,
  'reply_count': 127,
  'quote_count': 6,
  'bookmark_count': 2},
 '1805888653006139633': {'created_at': 'Wed Jun 26 08:59:41 +0000 2024',
  'view_count': '108423',
  'favorite_count': 653,
  'retweet_count': 111,
  'reply_count': 630,
  'quote_count': 52,
  'bookmark_count': 18},
 '1803820860223955110': {'created_at': 'Thu Jun 20 16:03:01 +0000 2024',
  'view_count': '48643',
  'favorite_count': 219,
  'retweet_count': 50,
  'reply_count': 364,
  'quote_count': 8,
  'bookmark_count': 5},
 '1803763951819162081': {'created_at': 'Thu Jun 20 12:16:53 +0000 2024',
  'view_count': '242685',
  'favorite_count': 490,
  'retweet_count': 126,
  'reply_count': 1683,
  'quote_count': 156,
  'bookmark_count': 50},
 '1803494697861783731': {'created_at': 'Wed Jun 19 18:26:57 +0000 2024',
  'view_count': '54839',
  'favorite_count': 161,
  'retweet_count': 3

In [48]:
tweet_len = [len(politician_tweet_data[politician]) for politician in politician_tweet_data]

In [49]:
np.min(tweet_len), np.max(tweet_len), np.mean(tweet_len), np.median(tweet_len), np.quantile(tweet_len, [0, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1.0])

(np.int64(0),
 np.int64(850),
 np.float64(708.6912751677852),
 np.float64(775.0),
 array([  0.  , 699.  , 775.  , 805.  , 834.2 , 838.  , 844.52, 850.  ]))

In [51]:
politician_tweet_dates = {}

for pol in politician_tweet_data:
    if len(politician_tweet_data[pol]) == 0:
        continue

    dates = [politician_tweet_data[pol][tweet]['created_at'] for tweet in politician_tweet_data[pol]]
    politician_tweet_dates[pol] = {'min_date': min(dates), 'max_date': max(dates)}


In [52]:
start_2023 = date(2023, 1, 1)
# end_2023   = date(2023, 12, 31)
# start_2024 = date(2024, 1, 1)
end_2024   = date(2024, 12, 31)

count = 0

for pol, dates in politician_tweet_dates.items():
    min_d = datetime.strptime(dates['min_date'], "%a %b %d %H:%M:%S %z %Y").date()
    max_d = datetime.strptime(dates['max_date'], "%a %b %d %H:%M:%S %z %Y").date()

    if start_2023 < min_d < end_2024:
        count += 1

count

112

In [61]:
start_2023 = date(2023, 1, 1)
# end_2023   = date(2023, 12, 31)
# start_2024 = date(2024, 1, 1)
end_2024   = date(2024, 12, 31)

for pol in politician_tweet_data:
    if pol not in politician_tweet_dates:
        # print(pol)
        continue

    dates = politician_tweet_dates[pol]
    min_d = datetime.strptime(dates['min_date'], "%a %b %d %H:%M:%S %z %Y").date()
    max_d = datetime.strptime(dates['max_date'], "%a %b %d %H:%M:%S %z %Y").date()

    if start_2023 < min_d < end_2024:
        for k, tweet in politician_tweet_data[pol].items():
            if tweet['view_count'] == -1:
                print(pol, tweet)
                pass


44177383 {'created_at': 'Fri Jul 19 14:23:58 +0000 2024', 'view_count': -1, 'favorite_count': 0, 'retweet_count': 98, 'reply_count': 0, 'quote_count': 0, 'bookmark_count': 0}
44177383 {'created_at': 'Fri Jul 12 17:49:50 +0000 2024', 'view_count': -1, 'favorite_count': 0, 'retweet_count': 52, 'reply_count': 0, 'quote_count': 0, 'bookmark_count': 0}
44177383 {'created_at': 'Thu Jul 11 17:07:08 +0000 2024', 'view_count': -1, 'favorite_count': 0, 'retweet_count': 9, 'reply_count': 0, 'quote_count': 0, 'bookmark_count': 0}
44177383 {'created_at': 'Tue Jul 09 15:04:34 +0000 2024', 'view_count': -1, 'favorite_count': 0, 'retweet_count': 24, 'reply_count': 0, 'quote_count': 0, 'bookmark_count': 0}
44177383 {'created_at': 'Mon Jul 08 16:51:26 +0000 2024', 'view_count': -1, 'favorite_count': 0, 'retweet_count': 45, 'reply_count': 0, 'quote_count': 0, 'bookmark_count': 0}
44177383 {'created_at': 'Sat Jul 06 13:40:12 +0000 2024', 'view_count': -1, 'favorite_count': 0, 'retweet_count': 93, 'reply_c

In [70]:
def change_times_from_dict(data_dict, n_bkps=3, metric='retweet_count', boundary="start"):

    df = pd.DataFrame.from_dict(data_dict, orient="index").reset_index().rename(columns={"index": "tweet_id"})
    if 'created_at' not in df.columns or metric not in df.columns:
        return []
    else:
        df["created_at"] = pd.to_datetime(df["created_at"], format="%a %b %d %H:%M:%S %z %Y")

        if metric == 'view_count':
            df = df[df[metric] != -1]

        df = df.sort_values("created_at").reset_index(drop=True)

        values = df[metric].to_numpy()
        times = df["created_at"].to_numpy()

        algo = rpt.Binseg(model="l2").fit(values)
        bkps = algo.predict(n_bkps=n_bkps)
        cp = bkps[:-1]                    

        if boundary == "start":
            idx = cp                       
        elif boundary == "end":
            idx = [i-1 for i in cp]      
        else:
            raise ValueError("boundary must be 'start' or 'end'")

        return [times[i] for i in idx]


In [ ]:
fdates = {}
start_2023 = date(2023, 1, 1)
end_2024   = date(2024, 12, 31)

for pol in politician_tweet_data:
    dates = politician_tweet_dates.get(pol, None)
    if dates is None:
        continue
    min_d = datetime.strptime(dates['min_date'], "%a %b %d %H:%M:%S %z %Y").date()
    max_d = datetime.strptime(dates['max_date'], "%a %b %d %H:%M:%S %z %Y").date()

    if not (start_2023 < min_d < end_2024):
        continue

    cdates = change_times_from_dict(politician_tweet_data[pol], n_bkps=1, metric='favorite_count', boundary="start")
    if len(cdates) > 0:
        for d in cdates:
            cdate = d.strftime("%m-%Y")
            if cdate not in fdates:
                fdates[cdate] = []
            fdates[cdate].append(reverse_user_id_maps[pol])

fdates
fdates = dict(sorted(fdates.items(), key=lambda x: len(x[1]), reverse=True))
fdates

In [105]:
interested_pols = ['AdamKinzinger',
 'mattgaetz',
 'tedcruz',
 'AndrewYang',
 'mtgreenee',
 'KariLake',
 'LindseyGrahamSC',
 'GovMikeHuckabee',
 'KamalaHarris',
 'Sen_JoeManchin',
 'RandPaul',
 'timkaine',
 'MeghanMcCain',
 'SenatorDurbin',
 'lisamurkowski',
 'SenRickScott',
 'realDonaldTrump',
 'GregAbbott_TX',
 'JulianCastro',
 'GabbyGiffords',
 'CoryBooker',
 'laurenboebert',
 'Jim_Jordan',
 'RashidaTlaib',
 'Mike_Pence',
 'JDVance',
 'SarahPalinUSA',
 'PeteButtigieg',
 'DouglasEmhoff',
 'JoshShapiroPA',
 'newtgingrich',
 'JoeBiden',
 'GavinNewsom',
 'votetimscott',
 'RealPNavarro',
 'Tim_Walz',
 'SenTomCotton']

politicians_party = {
    "AdamKinzinger": "Republican", 
    "mattgaetz": "Republican",
    "tedcruz": "Republican",
    "AndrewYang": "Other/Independent",  
    "mtgreenee": "Republican",
    "KariLake": "Republican",
    "LindseyGrahamSC": "Republican",
    "GovMikeHuckabee": "Republican",
    "KamalaHarris": "Democrat",
    "Sen_JoeManchin": "Democrat",
    "RandPaul": "Republican",
    "timkaine": "Democrat",
    "MeghanMcCain": "Republican",  
    "SenatorDurbin": "Democrat",
    "lisamurkowski": "Republican",
    "SenRickScott": "Republican",
    "realDonaldTrump": "Republican",
    "GregAbbott_TX": "Republican",
    "JulianCastro": "Democrat",
    "GabbyGiffords": "Democrat",
    "CoryBooker": "Democrat",
    "laurenboebert": "Republican",
    "Jim_Jordan": "Republican",
    "RashidaTlaib": "Democrat",
    "Mike_Pence": "Republican",
    "JDVance": "Republican",
    "SarahPalinUSA": "Republican",
    "PeteButtigieg": "Democrat",
    "DouglasEmhoff": "Democrat", 
    "JoshShapiroPA": "Democrat",
    "newtgingrich": "Republican",
    "JoeBiden": "Democrat",
    "GavinNewsom": "Democrat",
    "votetimscott": "Republican",
    "RealPNavarro": "Republican",  
    "Tim_Walz": "Democrat",
    "SenTomCotton": "Republican"
}

In [108]:
fdates = {}
start_2023 = date(2023, 1, 1)
end_2024   = date(2024, 12, 31)

for pol in politician_tweet_data:
    if reverse_user_id_maps[pol] not in interested_pols:
        continue
    dates = politician_tweet_dates.get(pol, None)
    if dates is None:
        continue
    min_d = datetime.strptime(dates['min_date'], "%a %b %d %H:%M:%S %z %Y").date()
    max_d = datetime.strptime(dates['max_date'], "%a %b %d %H:%M:%S %z %Y").date()

    if not (start_2023 < min_d < end_2024):
        continue

    cdates = change_times_from_dict(politician_tweet_data[pol], n_bkps=1, metric='favorite_count', boundary="start")
    if len(cdates) > 0:
        for d in cdates:
            cdate = d.strftime("%m-%Y")
            if cdate not in fdates:
                fdates[cdate] = []
            fdates[cdate].append(reverse_user_id_maps[pol])

fdates
fdates = dict(sorted(fdates.items(), key=lambda x: len(x[1]), reverse=True))
fdates

{'11-2024': ['AdamKinzinger',
  'mattgaetz',
  'tedcruz',
  'AndrewYang',
  'mtgreenee',
  'KariLake',
  'LindseyGrahamSC',
  'GovMikeHuckabee',
  'KamalaHarris',
  'Sen_JoeManchin',
  'RandPaul',
  'timkaine',
  'MeghanMcCain',
  'SenatorDurbin',
  'lisamurkowski',
  'SenRickScott',
  'realDonaldTrump',
  'GregAbbott_TX',
  'JulianCastro'],
 '07-2024': ['GabbyGiffords',
  'CoryBooker',
  'laurenboebert',
  'Jim_Jordan',
  'RashidaTlaib',
  'Mike_Pence',
  'JDVance',
  'SarahPalinUSA',
  'PeteButtigieg',
  'DouglasEmhoff',
  'JoshShapiroPA',
  'newtgingrich',
  'JoeBiden',
  'GavinNewsom',
  'votetimscott',
  'RealPNavarro',
  'Tim_Walz',
  'SenTomCotton']}

In [111]:
for k, v in fdates.items():
    rcount, dcount = 0, 0

    for pol in v:
        if politicians_party[pol] == 'Republican':
            rcount += 1
        elif politicians_party[pol] == 'Democrat':
            dcount += 1
    
    print(k, rcount, dcount)

11-2024 13 5
07-2024 9 9


In [134]:
for k, v in fdates.items():

    for pol in v:
        before_values, after_values = get_before_after_values(politician_tweet_data, user_id_maps[pol], 'favorite_count', [int(k.split('-')[1]), int(k.split('-')[0]), 1])
        s, p = mannwhitneyu(before_values, after_values, alternative='less')

        if p > 0.05:
            print(k, pol, 'No significant increase', len(before_values), len(after_values), p)


    print('---')

11-2024 AdamKinzinger No significant increase 76 171 0.09059991183099952
11-2024 KariLake No significant increase 313 480 0.1492307255955836
11-2024 SenatorDurbin No significant increase 569 174 0.23248053318440098
11-2024 JulianCastro No significant increase 425 12 0.057149508789045296
---
07-2024 RashidaTlaib No significant increase 299 506 0.962510038341415
07-2024 Mike_Pence No significant increase 262 527 0.9835649244421527
07-2024 JoeBiden No significant increase 541 279 0.19773908793102868
07-2024 GavinNewsom No significant increase 645 177 0.05607519949719461
07-2024 RealPNavarro No significant increase 566 251 0.8860179384338502
---


In [126]:
def get_before_after_values(politician_tweet_data, polid, metric, change_date):

    before_values = []
    after_values = []

    for tweet, data in politician_tweet_data[polid].items():
        created_at = data['created_at']
        created_at = datetime.strptime(created_at, "%a %b %d %H:%M:%S %z %Y")

        cyear, cmonth, cdate = change_date[0], change_date[1], change_date[2]
        if created_at < datetime(cyear, cmonth, cdate, tzinfo=created_at.tzinfo):
            before_values.append(data[metric])

        elif created_at >= datetime(cyear, cmonth, cdate, tzinfo=created_at.tzinfo):
            after_values.append(data[metric])

    return before_values, after_values

In [124]:
s, p = mannwhitneyu(before_values, after_values, alternative='less')
s, p

(np.float64(1183.5), np.float64(0.009848461757521939))

In [122]:
len(before_values), len(after_values)

(692, 7)

In [139]:
required_data = {}

for pol in politician_tweet_data:
    if reverse_user_id_maps[pol] not in interested_pols:
        continue
    dates = politician_tweet_dates.get(pol, None)
    if dates is None:
        continue
    min_d = datetime.strptime(dates['min_date'], "%a %b %d %H:%M:%S %z %Y").date()
    max_d = datetime.strptime(dates['max_date'], "%a %b %d %H:%M:%S %z %Y").date()

    if not (start_2023 < min_d < end_2024):
        continue

    required_data[pol] = politician_tweet_data[pol]



In [140]:
len(required_data)

37

In [145]:
with open('/Users/brahmaninutakki/saarland/twitter_extension/data/coding_assignment/politcians_twitter_data.json', 'w') as fp:
    json.dump(required_data, fp)

In [146]:
with open('/Users/brahmaninutakki/saarland/twitter_extension/data/coding_assignment/politcians_parties_map.json', 'w') as fp:
    json.dump(politicians_party, fp)

In [148]:
required_id_maps = {}

for pol in user_id_maps:
    if user_id_maps[pol] in required_data:
        required_id_maps[pol] = user_id_maps[pol]

In [155]:
with open('/Users/brahmaninutakki/saarland/twitter_extension/data/coding_assignment/politcians_name_twitterid_map.json', 'w') as fp:
    json.dump(required_id_maps, fp)